# Tutorial 7: Authoring APIs

In this tutorial, we'll do a more systematic walkthrough over the core authoring
APIs that you can use to define your workloads. Since this is a Python Jupyter
notebook, we'll default to Python, but the core abstractions are available also
in C++ and Java, albeit the exact syntax may vary. We encourate you to briefly
skim over the
[API documentation](https://github.com/google/genc/tree/master/genc/docs/api.md)
for an overview of the types of
APIs that exist in GenC, and how they vary across languages. In this tutorial,
our focus will be more hands-on and example-driven.

In the rest of the tutorial, we'll repeat some of the introductory material to
make it self-contained.

## Before we begin

As usual, make sure that you have a core development environment setup,
similarly to the preceding tutorials, so that you can load this notebook in
Jupyter and execute the code cells. For this, follow the basic instructions in
[SETUP.md](https://github.com/google/genc/tree/master/SETUP.md),
and reopen this notebook in Jupyter, and then execute the following code cells
to confirm that the setup works as expected.

In [ ]:
import genc

from genc.python import authoring
from genc.python import examples
from genc.python import interop
from genc.python import runtime

## Defining and running your first computation

Now, it's time to define your first computation. We're going to start with a
simple "hello, world" sort of example, a computation that uses a prompt
template to generate a greeting. You may write something as follows:

In [ ]:
@genc.python.authoring.traced_computation
def say_hello(name):
  return genc.python.authoring.prompt_template['Hello, {name}!'](name)

In order to run your computation, you're going to need to create an executor.
There are many ways to do it, but perhaps the simplest way to go is to register
a default executor we created, which enables you to call computations like the
one you've defined above similarly to how you'd call a Python function. You can
do this as follows:

In [ ]:
genc.python.examples.executor.set_default_executor()

Now you can run your computation:

In [ ]:
say_hello('Barbara')

'Hello, Barbara!'

You should see:

```
'Hello, Barbara!'
```

You can print the
[portable IR](https://github.com/google/genc/tree/master/genc/docs/ir.md)
that defines the computation you've just defined, as follows:

In [ ]:
print(say_hello.portable_ir)

lambda {
  parameter_name: "arg"
  result {
    block {
      local {
        name: "v_0"
        value {
          intrinsic {
            uri: "prompt_template"
            static_parameter {
              str: "Hello, {name}!"
            }
          }
        }
      }
      result {
        call {
          function {
            reference {
              name: "v_0"
            }
          }
          argument {
            reference {
              name: "arg"
            }
          }
        }
      }
    }
  }
}



With the IR at hand, there are more mechanisms at your disposal, such as running
your computation in a different programming language or execution environment.
These topics are outside of the scope of this authoring tutorial, but please see
[api.md](https://github.com/google/genc/tree/master/genc/docs/api.md)
for details if you're interested.

## The anatomy of your first computation

Now, let's take a closer look at the computation you've defined, and use it as
the opportunity to explain a few core concepts. As a reminder, here's what it
looked like:

```
@genc.python.authoring.traced_computation
def say_hello(name):
  return genc.python.authoring.prompt_template['Hello, {name}!'](name)
```

The first thing to note is that, you've defined it using a syntax similar to a
Python function, with a `traced_computation` decorator. The computation under
the hood is not actually Python code (as you saw above by printing the
`portable_ir` attribute, it's actually in the IR form), but it's constructed in
a Python-like fashion. In essence, code inside the decorated Python function
constructs the computation "graph" (so to speak), with each Python-like function
call inside adding a piece of that graph. The Python code inside it is being
traced at declaration time. In case you're familiar with the "graph mode" in
TensorFlow, the mechanism we use here is very similar.

Thus, in general, a definition of a computation using the tracing API that we're
showing here is going to look like the following. It can take one or more
arguments, which you can then refer to in the body of the computation. At some
point, you must use the `return` statement to indicate the result.

```
@genc.python.authoring.traced_computation
def say_hello(arg_1, arg_2, ..., arg_n):
  # Some code that might refer to any of the `arg_1`, `arg_2`, ..., `arg_n`
  return ...
```

In your example, you've also seen the use of a `prompt_template`, of the
building blocks we provide. Like most building blocks, it takes two parameters,
one in square brackets - this is called "static parameter", a baked-in
part of the computation logic, and one in round brackets - this is called the
"dynamic parameter", and it can be computed while your computation runs. In this
case, the prompt template text is static, but the name being fed to it is
dynamic, as it's supplied whren the computation is being invoked.

## Playing with prompt templates

Now that you understand the syntax, let's play with more this initial example,
still just using the prompt template as a building block, and illustrate a few
concepts in the process.

First, let's throw in another prompt template, so that your computation now
consists of two elements - the two prompt templates chained together, with one
feeding into the other:

In [ ]:
@genc.python.authoring.traced_computation
def say_hello(x):
  friendly_name = genc.python.authoring.prompt_template['my friend {name}'](x)
  return genc.python.authoring.prompt_template['Hello, {name}!'](friendly_name)

say_hello('Barbara')

'Hello, my friend Barbara!'

You should see:

```
'Hello, my friend Barbara!'
```

First, notice that the names of the parameters to your computation and the
names of the parameters iun the prompt template are separate - there's no
connection between them:
*   The name `x` in the body of the computation represents the argument.
*   That argument `x` is being fed as an input into the first prompt template.
*   In the text of the first prompt template, the input is represented as a
    placeholders string `{name}`. Here, `{name}` simply represents the input.
    Any string will do.
*   We capture the output produced by the first promtp template, and give it a
    name `friendly_name` in the body of the traced computation.
*   This is subsequently fed as input to the second prompt template call.
*   The text of the second prompt template, again, happens to use `{name}` to
    represents its input. Any string would do.
*   The result of the second prompt template (and thus the entire chain) is
    being returned as the result of the computation.

The overall flow here is meant to resemble ordinary programming in Python, where
you intersperse function calls, and assign results to local variables (but keep
in mind the underlying logic is not Python, which has some implications, e.g.,
in terms of the order of execution - mor on this shortly).

With this, we're ready to expand on our example. Let's throw in an additional
argument to your function, and upgrade to use multi-variate prompt templates,
i.e., prompt templates that can take more than one parameter as input.

In [ ]:
@genc.python.authoring.traced_computation
def fancy_say_hello(title, first_name, last_name):
  full_name = genc.python.authoring.prompt_template_with_parameters[
      'my friend {x} {y}', ['x', 'y']](first_name, last_name)
  return genc.python.authoring.prompt_template_with_parameters[
      'Hello, {a}! Can I call you {b}? Or would you prefer {c} {d}?',
       ['a', 'b', 'c', 'd' ]](full_name, first_name, title, last_name)

fancy_say_hello('Sir', 'Alexander', 'Fleming')

'Hello, my friend Alexander Fleming! Can I call you Alexander? Or would you prefer Sir Fleming?'

You should see:

```
'Hello, my friend Alexander Fleming! Can I call you Alexander? Or would you
prefer Sir Fleming?'
```

In the above example, we used building block `prompt_template_with_parameters`,
which takes the additional list of parameter names. This list helps determine
the order in which parameters will be given, which may differ from the order in
which they're used in the text. Also, the same parameter can appear in the text
multiple times, like in this example:

In [ ]:
@genc.python.authoring.traced_computation
def show_off_counting(x, y, z):
  return genc.python.authoring.prompt_template_with_parameters[
      'I can count forward, check this out: {a}, {b}, {c}. And backwards, too: {c}, {b}, {a}.',
       ['a', 'b', 'c']](x, y, z)

show_off_counting('one', 'two', 'three')

'I can count forward, check this out: one, two, three. And backwards, too: three, two, one.'

You should see:

```
'I can count forward, check this out: one, two, three. And backwards, too: three, two, one.'
```

## Model inference calls

Model calls are another basic building block. Like other building blocks, they
are available in the `genc.authoring` namespace. Here's an example of
how you can call a built-in test model, and compose it in a chain with a prompt
template.

In [ ]:
@genc.python.authoring.traced_computation
def simple_chain(x):
  y = genc.python.authoring.prompt_template['Tell me about {topic}.'](x)
  return genc.python.authoring.model_inference['test_model'](y)

simple_chain('scuba diving')

'This is an output from a test model in response to "Tell me about scuba diving.".'

You should see:

```
This is an output from a test model in response to "Tell me about scuba diving.".
```

Now, try this with a real (non-test) model. There are a number of models you can use - the exact selection will
depend on how your environment is configured. In this
tutorial, we'll use the Gemini model. You'll need the API
key, e.g., from Google AI Studio (see
[instructions](https://ai.google.dev/tutorials/rest_quickstart)). Copy the key in place of the commented
out text below:

In [ ]:
api_key = "" # Type your API key here

In [ ]:
@genc.python.authoring.traced_computation
def gemini(x):
  model = genc.python.authoring.model_inference_with_config[{
      'model_uri': '/cloud/gemini',
      'config': genc.python.interop.gemini.create_config(api_key)}]
  return model(x)

gemini('What color is the ocean? Answer in one word only.')

'Blue'

You should see:
```
'Blue'
```

## Composing larger computations from simpler ones

So far, each computation you defined existed on its own, but you can use any
of your computations as a building block in another computation, in the same
manner as how you used prompt templates and inference calls. Here's an example
based on computations you defined earlier in this tutorial.

In [ ]:
@genc.python.authoring.traced_computation
def greet_book_author(book_title):
  name_prompt = genc.python.authoring.prompt_template[
      'Return the first and last name of the author of a book entitled "{x}".']
  author = gemini(name_prompt(book_title))
  return say_hello(author)

greet_book_author('The Old Man and the Sea')

'Hello, my friend Ernest Hemingway!'

You should see:

```
Hello, my friend Ernest Hemingway!
```

## Concurrency and order of execution

As you recall, we mentioned that we use Python-like syntax, but the code
written in GenC is not Python - it's represented as a portable IR, and
executed by a C++ runtime.

The order in which code excutes is determined only by how data flows from
one statement to another, not by the order in which statements in the code
are written. If a pair of statements doesn't depend on one-another, they
will execute in parallel.

Here's an example of a computation that makes 3 concurrent calls to Gemini:

In [ ]:
@genc.python.authoring.traced_computation
def get_attribute_from_gemini(the_thing, its_attribute):
  return gemini(genc.python.authoring.prompt_template_with_parameters[
      'What is the {x} of {y}?', ['x', 'y']](
      its_attribute, the_thing))

@genc.python.authoring.traced_computation
def tell_me_about(x):
  color = get_attribute_from_gemini(x, 'color')
  taste = get_attribute_from_gemini(x, 'taste')
  price = get_attribute_from_gemini(x, 'price')
  return genc.python.authoring.prompt_template_with_parameters[
      'Here is some info on {a}: {b} {c} {d}', ['a', 'b', 'c', 'd']](
          x, color, taste, price)

tell_me_about('cucumbers')

'Here is some info on cucumbers: Cucumbers can have different colors depending on the variety, but the most common color is **dark green**. Refreshing, crisp, mildly sweet, slightly bitter, vegetal, grassy The price of cucumbers can vary depending on factors such as location, season, and market conditions. Generally, the price of a single cucumber can range from $0.25 to $1.00, while a pound of cucumbers can cost anywhere from $1.00 to $2.50.'

What you'll get may vary, but here's a sample output:

```
Here is some info on cucumbers: Cucumbers can vary in color, but the most common colors are:

* **Green:** This is the most common color for cucumbers, and it ranges from light green to dark green.
* **Yellow:** Some varieties of cucumbers have a yellow color, either when they are ripe or when they are still young.
* **White:** White cucumbers are less common, but they exist.
* **Red:** Some varieties of cucumbers have a reddish skin, which can be either a deep red or a more subtle pinkish hue.

Cucumbers have a mild, refreshing taste. They are slightly sweet and have a hint of bitterness. The skin of the cucumber is slightly bitter, while the flesh is more sweet. The taste of cucumbers can vary depending on the variety and growing conditions.

Cucumber prices vary widely depending on geographic location, season, and market conditions.

In the United States, the average price of a cucumber is between $0.25 and $0.75 each, or $1.50 to $4 per pound.

During the summer months, when cucumbers are in peak season, prices are typically lower. In the winter months, when cucumbers are imported from warmer climates, prices are usually higher.

For the most accurate and up-to-date pricing, it is best to check with your local grocery store or farmers' market.
```

## Basic control flow constructs: conditionals

Control flow looks a bit different than ordinary Python syntax, since at this
time we don't support autograph-like tracing. You express control flow in a
functional manner, by providing a Boolean input and a pair of functions, one to
evaluate in case the input is true, and the other otherwise.

Here's an example of how you can define a conditional. The "if" and "else"
branches go in the square brackets, and the condition is the dynamic parameter.

In [ ]:
@genc.python.authoring.traced_computation
def is_child(person):
  return genc.python.authoring.regex_partial_match['child'](gemini(
      genc.python.authoring.prompt_template[
      'Respond with the word "child" if {x} is a child, "adult" otherwise.'](
          person)))

@genc.python.authoring.traced_computation
def recommend_product(product_category):
  return gemini(genc.python.authoring.prompt_template[
      'Recommend a product of type "{x}". Answer concisely.'](product_category))

@genc.python.authoring.traced_computation
def recommend_product_for(person):
  return genc.python.authoring.conditional[
      recommend_product('toy'),
      recommend_product('book')](
          is_child(person))

recommend_product_for('my seven year old nephew')

'LEGO Star Wars Millennium Falcon'

In [ ]:
recommend_product_for('the president of the United States of America')

'"The Hitchhiker\'s Guide to the Galaxy" by Douglas Adams'

## Basic flow control constructs: loops

Continuing from the previous section, let's try writing some loops. There are
a number of loop-like constructs, perhaps the simplest of which is a "while"
loop. Similar in spirit to the conditional statement, it takes a pair of
functions in square brackets - the first being the loop "condition" function
to evaluate in each iteration on the loop state, and the second being the "body" to transform the loop state from before to after, and the initial loop
state goes in as the dynamic parameter.

Here's a counting example:

In [ ]:
@genc.python.authoring.traced_computation
def get_the_counter(x):
  return gemini(genc.python.authoring.prompt_template[
      'Return the counter that appears as the first element in "{x}".'](x))

@genc.python.authoring.traced_computation
def get_the_divisor(x):
  return gemini(genc.python.authoring.prompt_template[
      'Return the divisor that appears as the second element in "{x}".'](x))

@genc.python.authoring.traced_computation
def is_larger_than_one(x):
  return genc.python.authoring.regex_partial_match['YES'](gemini(
      genc.python.authoring.prompt_template[
      'Respond with the word "YES" if {x} is larger than one, "NO" otherwise. Just one word, "YES" or "NO".'](x)))

@genc.python.authoring.traced_computation
def divide_by_two(x):
  return gemini(genc.python.authoring.prompt_template[
      'Return {x} divided by two (as an integer). Just the number, no reasoning.'](x))

@genc.python.authoring.traced_computation
def add_one(x):
  return gemini(genc.python.authoring.prompt_template['Return {x} plus one. Just the number, no reasoning.'](x))

@genc.python.authoring.traced_computation
def should_continue(loop_state):
  return is_larger_than_one(get_the_divisor(loop_state))

@genc.python.authoring.traced_computation
def advance_the_loop(loop_state):
  return genc.python.authoring.prompt_template_with_parameters[
      'The counter is "{x}", and the divisor is "{y}".', ['x', 'y']](
          add_one(get_the_counter(loop_state)),
          divide_by_two(get_the_divisor(loop_state)))

# TODO: Fix this clash with the Python language keyword.
while_loop = getattr(genc.python.authoring, 'while')

@genc.python.authoring.traced_computation
def compute_logarithm(number):
  initial_state = genc.python.authoring.prompt_template[
      'The counter is "0", and the divisor is "{x}".'](number)
  end_state = while_loop[should_continue, advance_the_loop](initial_state)
  return get_the_counter(end_state)

compute_logarithm('10')

'3'

You should see:

```
'3'
```

Keep in mind that if you're going to make a lot of model calls, on occasion you
may exceed the allowed limit, which could cause code like the above to fail.
Usualluy retrying helps. To make the code more robust, you'd want to add error
handling for aberrant conditions (such as empty model output, an error message,
etc.).

## Parting words

We've reached the end of the tutorial - you now have the basic faimilarity with
the GenC authoring syntax and some of the core constructs. For more, consider
reviewing other tutorials and reading through the API documentation. If there's
a building block you'd like to see that you can't find, please reach out, or
considering contributing. Good luck!
